In [2]:
import faiss
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaLLM

# Load and split the PDF document
file_path = "C:\\Users\\HP\\test_bot\\data\\NMAM_Manual.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

# Concatenate the text from all pages
raw_text = " ".join([page.page_content for page in pages if page.page_content])

# Split text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
text_chunks = text_splitter.split_text(raw_text)

# Create Document objects for each chunk
docs = [Document(page_content=chunk) for chunk in text_chunks]

# Initialize HuggingFace embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS index from document embeddings
library = FAISS.from_documents(docs, embeddings)


 #Function to search FAISS index and return the most relevant text chunk
def search_faiss_index(library, question):
    question_embedding = embeddings.embed_query(question)
    D, I = library.index.search(np.array([question_embedding]), 1)  # k=1 for the closest match
    return docs[I[0][0]].page_content

# Function to refine answer with Ollama LLM
def refine_answer_with_ollama(model, context, question):
    template = """
    Answer the question below.

    Here is the conversation history: {Context}

    Question: {question}

    Answer:
    """
    input_prompt = template.format(Context=context, question=question)
    result = model.invoke(input=input_prompt)
    return result

# Answer a question based on the PDF content
question = "hi"
context = search_faiss_index(library, question)

# Initialize the Ollama LLM model
model = OllamaLLM(model='llama3.2')

# Refine the answer using Ollama LLM
refined_answer = refine_answer_with_ollama(model, context, question)

#print("Question:", question)
print("Answer:", refined_answer)

ModuleNotFoundError: No module named 'langchain_community'